In [1]:
from transformers import pipeline
import cv2
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import nltk
import os
from google.cloud import speech_v1 as speech
from google.cloud import storage
import io
from pydub import AudioSegment
import math
from tqdm import tqdm
from transformers import MT5Tokenizer, MT5ForConditionalGeneration
import sentencepiece

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "interview-400010-9abf1ded3113.json"

def extract_keywords(text):
    stop_words = set(stopwords.words('english')) | set(stopwords.words('korean'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.isalnum() and word.lower() not in stop_words]
    keyword_freq = Counter(filtered_words)
    return keyword_freq.most_common(10)

def generate_summary(text, max_length):
    tokenizer = MT5Tokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
    model = MT5ForConditionalGeneration.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
    summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

    max_input_length = 512  # mT5 모델의 최대 입력 길이

    # Tokenize the input text and split it into chunks of max_input_length tokens
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding="longest").input_ids[0]
    chunks = [tokens[i:i + max_input_length] for i in range(0, len(tokens), max_input_length)]

    summaries = []
    for chunk in tqdm(chunks, desc="Summarizing chunks"):
        chunk_text = tokenizer.decode(chunk, skip_special_tokens=True)
        summary_max_length = max(5, min(max_length, 512))
        summary_min_length = max(1, int(summary_max_length * 0.7))
        summary = summarizer(chunk_text, max_length=summary_max_length, min_length=summary_min_length, do_sample=False)[0]['summary_text']
        summaries.append(summary)

    return " ".join(summaries)

def convert_to_vertical(input_path, output_path):
    cap = cv2.VideoCapture(input_path)

    if not cap.isOpened():
        print(f"Error: Unable to open video file {input_path}")
        return

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (1080, 1920))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        height, width, _ = frame.shape
        if width > height:
            new_height = int((width / 9) * 16)
            top = (new_height - height) // 2
            bottom = new_height - height - top
            frame = cv2.copyMakeBorder(frame, top, bottom, 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        else:
            new_width = int((height / 16) * 9)
            left = (new_width - width) // 2
            right = new_width - width - left
            frame = cv2.copyMakeBorder(frame, 0, 0, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))

        out.write(frame)

    cap.release()
    out.release()

def transcribe_audio_chunk(audio_chunk, language_code, sample_rate):
    client = speech.SpeechClient()
    audio = speech.RecognitionAudio(content=audio_chunk)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=sample_rate,
        language_code=language_code,
    )

    response = client.recognize(config=config, audio=audio)

    transcript = ""
    for result in response.results:
        transcript += result.alternatives[0].transcript
    return transcript

def transcribe_audio(audio_path, language_code):
    audio = AudioSegment.from_file(audio_path)
    audio = audio.set_channels(1)  # 오디오를 모노로 변환
    sample_rate = audio.frame_rate
    chunk_size = 10000  # 10 seconds per chunk
    total_chunks = math.ceil(len(audio) / chunk_size)
    transcript = ""

    for i in tqdm(range(total_chunks), desc="Transcribing audio"):
        start = i * chunk_size
        end = start + chunk_size
        audio_chunk = audio[start:end]

        audio_chunk.export("temp_chunk.wav", format="wav")
        with io.open("temp_chunk.wav", "rb") as audio_file:
            content = audio_file.read()
            transcript += transcribe_audio_chunk(content, language_code, sample_rate)

    return transcript

def process_video(file_path, language_code, duration, output_video_path):
    transcript = transcribe_audio(file_path, language_code)
    print("Transcript:", transcript[:500])  # 처음 500자만 출력
    summary = generate_summary(transcript, max_length=duration)
    print("Summary:", summary)
    convert_to_vertical(file_path, output_video_path)
    return summary, output_video_path


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jwoo3\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jwoo3\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# 개별 단계 테스트 함수
def test_transcribe_audio(file_path, language_code):
    transcript = transcribe_audio(file_path, language_code)
    print("Transcript:", transcript[:500])  # 처음 500자만 출력
    return transcript

def test_generate_summary(transcript, duration):
    summary = generate_summary(transcript, max_length=duration)
    print("Summary:", summary)
    return summary

def test_convert_to_vertical(file_path, output_video_path):
    convert_to_vertical(file_path, output_video_path)
    print("Converted video saved to:", output_video_path)
    
file_path = "videos/news1.mp4"  # 여기에 테스트할 비디오 파일 경로를 넣으세요.
language_code = "ko-KR"  # 한국어(Korean) 언어 코드
duration = 120  # 요약할 최대 길이 (초)
output_video_path = "videos/shorts_output.mp4"  # 출력 비디오 파일 경로

# Step 1: Transcribe audio
transcript = test_transcribe_audio(file_path, language_code)

# Step 2: Generate summary
summary = test_generate_summary(transcript, duration)

# Step 3: Convert to vertical video
test_convert_to_vertical(file_path, output_video_path)

print("Summary:", summary)
print("Output video path:", output_video_path)


Transcribing audio: 100%|██████████| 60/60 [02:24<00:00,  2.40s/it]


Transcript: 여러분 안녕하십니까 오늘은 저희가 단독 취재한 내용으로 뉴스 시작하겠습니다 다음 주 월요일부터는 병원에 갈 때신분증을 꼭 챙겨야 합니다 환자가 건강보험 자격이 있는지 혹시 다른 사람 명의로 진료받는 건 아닌지 병원이 확인하기 위해서입니다신분증 대신 휴대전화에 모바일 건강보험증을 설치해서 그걸 병원에 보여 줘도 됩니다 그런데이 모바일다른 사람의 휴대전화 해도 쉽게 설치할 수 있고 또 그걸 병원이 적발하기 어렵다는 사실이 저희 취재 결과 확인됐습니다기자의 단독 보도입니다 신분증 지참 필수 란 안내 포스터가 붙은 한내과의원 의원 협조를 미리 받아모바일 건강보험증으로 진료 접수를 해 봤습니다 건강보험증 qr 코드를 병원 기기로 인식하자 건강 보험 자격 확인이 되고 문제 없이완료됐습니다 하지만 기자가 제시한 모바일 건강보험증은 본인이 아닌 동료의 것입니다 타인 명의의 건강보험증을 병원에 제출했는데걸러내지 못한 겁니다 어떻게 이게 가능한걸까 모바일 건강보험증 앱은 휴대 전화 번호를 입력한뒤 인


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Summarizing chunks: 100%|██████████| 5/5 [00:55<00:00, 11.19s/it]


Summary: 오늘은 영국 공영방송 BBC 뉴스의 한국어 라디오, BBC코리아 방송의 2018년 5월 29일 보도입니다 환자가 건강보험 자격이 있는지 혹시 다른 사람 명의로 진료받는 건 아닌지 확인하기 위해서 모바일 건강 보험증을 설치해서 그걸 병원에 보여 줘도 될 수 있다 신종 코로나바이러스 감염증(코로나19) 사태로 인한 본인 강화 제도가 다음 주부터 시행될 것으로 보인다 통신사 인증을 둘러싼 논란이 일고 있다 이에 대한 대책마련이 필요하다고 보건복지부 관계자가 말했다 그러나 이런 내용이 없으면 서울에서 아파트 전세 구하기가 갈수록 어려워지면서 전셋값이계속 오르고 있는 것으로 나타났다 공급 부족에 대한 고민이 커지고 있기 때문에 이번 주 도쿄올림픽 개최를 앞두고 이호건 기자가 집회 후네 번째 긴 상승을 기록했다 지난해 제주에서 시작된 벌마늘 피해가 전남 경남 등 등으로 빠르게 확산하면서 정부는 농업 죄로 인정하고 재난지원금 지급에 나섰습니다 올해 국산 마늘 생산량은 평년보다 줄어들 수밖에 없는 상황 이번 달이면 파로 농산물의 가격이 좀 이중섭 전 국방부장관이 4차혁명 공판에서 증인으로 불러 달라는 박대령 박병직 요청을 받고 오늘 증인 채택 한 지은이는으로 재판에 나가서 증언 하겠다고 밝혔습니다 대통령은 언제 판이 없어 특검법 수영을 초과했기 때문에 장관은
Converted video saved to: videos/shorts_output.mp4
Summary: 오늘은 영국 공영방송 BBC 뉴스의 한국어 라디오, BBC코리아 방송의 2018년 5월 29일 보도입니다 환자가 건강보험 자격이 있는지 혹시 다른 사람 명의로 진료받는 건 아닌지 확인하기 위해서 모바일 건강 보험증을 설치해서 그걸 병원에 보여 줘도 될 수 있다 신종 코로나바이러스 감염증(코로나19) 사태로 인한 본인 강화 제도가 다음 주부터 시행될 것으로 보인다 통신사 인증을 둘러싼 논란이 일고 있다 이에 대한 대책마련이 필요하다고 보건복지부 관계자가 말했다 그러나 이런 내용이 없으면 서울에서 아파